In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Carregamento da listagem de propostas
listagem = pd.DataFrame(pd.read_excel('../Listagem/listagem_12.xlsx'))
#listagem

In [ ]:
# Limpeza de colunas vazias
listagem = listagem.drop(listagem.columns[[1,3,5,7,9]], axis=1)

#listagem = listagem.drop('Unnamed: 1', axis=1)
#listagem = listagem.drop('Unnamed: 3', axis=1)
#listagem = listagem.drop('Unnamed: 5', axis=1)
#listagem = listagem.drop('Unnamed: 7', axis=1)
#listagem = listagem.drop('Unnamed: 9', axis=1)

In [ ]:
# Limpeza de linhas varias (NaN) - A SER FEITO
listagem = listagem.dropna()


In [ ]:
display(listagem)